# LlamaParse - Parsing Complex Documents

With the release of [LlamaParse](https://github.com/run-llama/llama_parse) and [LlamaCloud](https://cloud.llamaindex.ai), LlamaIndex is demonstrating the next step of evolution for its offerings!

From the repository:

> LlamaParse is an API created by LlamaIndex to efficiently parse and represent files for efficient retrieval and context augmentation using LlamaIndex frameworks.

What LlamaIndex has done is created an API Endpoint that we can access (currently for free up to 10,000 pages of PDFs a day) that will parse out PDF files into either plain-text or markdown. That second one means we have a way to retain structural data that can be leveraged for more structural queries!

They've also [recently released](https://www.llamaindex.ai/blog/llamaindex-v0-10-838e735948f8) their v0.10 which, similar to LangChain's v0.1.0, provides some stability and methodological changes to move LlamaIndex into the production-ready space. (seeyah later `ServiceContext`!)

Let's dive in and see what we can do with this new tool!

## Load and Parse PDFs

We'll start, as always, by grabbing some dependencies.

In [2]:
!pip install -qU llama-index llama-parse ragas

In [3]:
import os
import getpass

os.environ["LLAMA_CLOUD_API_KEY"] = getpass.getpass("LLamaParse API Key:")

LLamaParse API Key:··········


Since we'll be using OpenAI as our LLM today - we'll need to pass that API key as well.

In [4]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

OpenAI API Key:··········


Let's make sure we can run async in our Colab instance.

In [5]:
import nest_asyncio

nest_asyncio.apply()

### LlamaParse Initialization

Here we can initialize our `LlamaParse` object.

Notice that there's a few parameters worth paying attention to:

- `result_type` - at time of writing this notebook the options are limited to `"text"` and `"markdown"`. Markdown will be our choice as it will retain structured information quite nicely.
- `num_workers` - this will let us set how many workers we'll need. Generally we'll want to set this to the number of files we're going to need to parse. (the maximum is `10`)

In [6]:
from llama_parse import LlamaParse

parser = LlamaParse(
    result_type="markdown",
    verbose=True,
    language="en",
    num_workers=2,
)

### Uploading Files

We'll next need to upload some files to test our the parser!

Let's use [NVIDIA's 10-K](https://d18rn0p25nwr6d.cloudfront.net/CIK-0001045810/1cbe8fe7-e08a-46e3-8dcc-b429fc06c1a4.pdf) and the [Office of Educational Technology's AI and the Future of Learning report](https://www2.ed.gov/documents/ai-report/ai-report.pdf).

You can upload them below - be careful to make sure the file matches with what you've uploaded!

In [7]:
from google.colab import files

ships_manual = files.upload()

Saving Ships_3m_manual_04790.008D.pdf to Ships_3m_manual_04790.008D.pdf


### Parsing Our Files

Now that we've uploaded our files and set-up our `LlamaParser` we're ready to parse some files!

Running this cell seems very inconsistent - with some files taking ~6min., and others taking ~4s. It seems there is some level of caching, but you can medium -> long wait times for this next cell.

> NOTE: As of time of writing, only `.pdf` files are accepted.

In [8]:
documents = parser.load_data(["/content/Ships_3m_manual_04790.008D.pdf"])

Started parsing the file under job_id 04746d42-5b43-4a5b-8b66-2a4f836e4951


In [9]:
print(documents[0])

Doc ID: 9bf0d0b0-03b4-4894-952e-953c011d2c42
Text: # NAVSEAINST 4790.8D  17 Jun 2021  MATERIAL  MM Ships' 3-M
Manual NAVSEA NAVAL SEA SYSTEMS COMMAND --- # DEPARTMENT OF THE NAVY
NAVAL SEA SYSTEMS COMMAND 1333 ISAAC HULL AVE SE WASHINGTON NAVY YARD
DC 20376-0001 IN REPLY REFER TO NAVSEAINST 4790.8D Ser 00/239  NAVSEA
INSTRUCTION 4790.8D 17 Jun 2021 From: Commander, Naval Sea Systems
Command Subj...


In [10]:
print(documents[0].text[:1000])

# NAVSEAINST 4790.8D

17 Jun 2021

MATERIAL

MM
Ships' 3-M
Manual
NAVSEA
NAVAL SEA SYSTEMS COMMAND
---
# DEPARTMENT OF THE NAVY
NAVAL SEA SYSTEMS COMMAND
1333 ISAAC HULL AVE SE
WASHINGTON NAVY YARD DC 20376-0001
IN REPLY REFER TO
NAVSEAINST 4790.8D
Ser 00/239

NAVSEA INSTRUCTION 4790.8D
17 Jun 2021
From: Commander, Naval Sea Systems Command
Subj: SHIPS’ MAINTENANCE AND MATERIAL MANAGEMENT MANUAL
Ref: (a) OPNAVINST 4790.4F
(b) COMUSFLTFORCOMINST 4790.3 REV D

1. Purpose. To issue a revised Ships’ Maintenance and Material Management (3-M) Manual
which incorporates new and updated maintenance management policy. This instruction is a
complete revision and should be reviewed in its entirety.

2. Cancellation. NAVSEAINST 4790.8C.

3. Responsibilities. Reference (a) contains Chief of Naval Operations (OPNAV) policy
regarding the 3-M System and assigns responsibilities to Naval Sea Systems Command
(NAVSEA) Field Activities, Systems Commands (SYSCOMS) and the Naval Medical
Command (NMC) for pro

## LlamaIndex Recursive Query Engine

Now that we have some parsed objects - let's see how well we can leverage them using one of the [example query engines](https://github.com/run-llama/llama_parse/blob/main/examples/demo_advanced.ipynb).

### Setting our...Settings

That's right! `ServiceContext` is dead, long live `Settings`.

Let's point our generic LLM to `gpt-3.5-turbo` and our generic embedding model as `text-embedding-3-small`.

> NOTE: You'll notice we're pulling `Settings` our of `llama_index.core` which is a major part of their `v0.10` update!

In [9]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

We're going to use a `MarkdownElementNodeParser` to help make sense of our Markdown objects so we can leverage the potentially structured information in the parsed documents.

- Check out the [docs](https://docs.llamaindex.ai/en/stable/api/llama_index.core.node_parser.MarkdownElementNodeParser.html)

In [10]:
from llama_index.core.node_parser import MarkdownElementNodeParser

node_parser = MarkdownElementNodeParser(llm=OpenAI(model="gpt-3.5-turbo"), num_workers=8)

Let's parse!

> NOTE: There appears to be inconsistent errors - but the parser is largely able to extract and understand structured data within the document provided by the parser

In [ ]:
nodes = node_parser.get_nodes_from_documents(documents=[documents[0]])

Now we can extract our `base_nodes` and `objects` to create our `VectorStoreIndex`.

In [12]:
base_nodes, objects = node_parser.get_nodes_and_objects(nodes)

In [ ]:
document_text = "\n".join([node.text for node in nodes])

In [ ]:
document_text

'NAVSEAINST 4790.8D\n\n17 Jun 2021\n\nMATERIAL\n\n(MM)\nShips\' 3-M\nManual\nNAVSEA\nNAVAL SEA SYSTEMS COMMAND\n---\n DEPARTMENT OF THE NAVY\nNAVAL SEA SYSTEMS COMMAND\n1333 ISAAC HULL AVE SE\nWASHINGTON NAVY YARD DC 20376-0001\nIN REPLY REFER TO\nNAVSEAINST 4790.8D\nSer 00/239\n\nNAVSEA INSTRUCTION 4790.8D\n17 Jun 2021\nFrom: Commander, Naval Sea Systems Command\nSubj: SHIPS’ MAINTENANCE AND MATERIAL MANAGEMENT MANUAL\nRef: (a) OPNAVINST 4790.4F\n(b) COMUSFLTFORCOMINST 4790.3 REV D\n\n1. Purpose. To issue a revised Ships’ Maintenance and Material Management (3-M) Manual\nwhich incorporates new and updated maintenance management policy. This instruction is a\ncomplete revision and should be reviewed in its entirety.\n\n2. Cancellation. NAVSEAINST 4790.8C.\n\n3. Responsibilities. Reference (a) contains Chief of Naval Operations (OPNAV) policy\nregarding the 3-M System and assigns responsibilities to Naval Sea Systems Command\n(NAVSEA) Field Activities, Systems Commands (SYSCOMS) and the

Let's build the index!

In [13]:
from llama_index.core import VectorStoreIndex

recursive_index = VectorStoreIndex(nodes=base_nodes+objects)

In [15]:
import pickle

file_path = "recursive_index.pkl"

# Open the file in binary write mode
with open(file_path, 'wb') as file:
    # Dump the object into the file using pickle
    pickle.dump(recursive_index, file)

print("Object saved successfully!")

Object saved successfully!


### Recursive Query Engine

Now we can build our Recursive Query Engine with reranking!

We'll need to do a few steps:

1. Initalize our reranker using `FlagEmbeddingReranker` powered by the `BAAI/bge-reranker-large`.
2. Set up our recursive query engine!

First, let's install some requirements.

In [16]:
!pip install -qU llama-index-postprocessor-flag-embedding-reranker git+https://github.com/FlagOpen/FlagEmbedding.git

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 8.4 MB/s eta 0:00:00


First up, we'll initialize our reranker - we'll be leveraging [this](https://github.com/FlagOpen/FlagEmbedding) repo to leverage our [`BAAI/bge-reranker-large`](https://huggingface.co/BAAI/bge-reranker-large).

Once that's done - we can follow a fairly standard flow of creating our query engine!

In [ ]:
ships_manual_nodes = node_parser.get_nodes_from_documents(documents=[documents[0]])

In [18]:
ships_base_nodes, ships_objects = node_parser.get_nodes_and_objects(ships_manual_nodes)

In [19]:
ships_recursive_index = VectorStoreIndex(nodes=ships_base_nodes + ships_objects, include_content=True)

In [20]:
from llama_index.postprocessor.flag_embedding_reranker import FlagEmbeddingReranker

reranker = FlagEmbeddingReranker(
    top_n=5,
    model="BAAI/bge-reranker-large",
)

ships_recursive_query_engine = ships_recursive_index.as_query_engine(
    similarity_top_k=15,
    node_postprocessors=[reranker],
    verbose=True
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/801 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

In [21]:
file_path = "ships_recursive_query_engine.pkl"

# Open the file in binary write mode
with open(file_path, 'wb') as file:
    # Dump the object into the file using pickle
    pickle.dump(ships_recursive_query_engine , file)

print("Object saved successfully!")

Object saved successfully!


In [22]:
query = "What is the alteration category for prefix TD?"
response = ships_recursive_query_engine.query(query)

Retrieval entering id_9204c751-aa43-4f11-909b-0a60f51cc466_1450_table: TextNode
Retrieving from object TextNode with query What is the alteration category for prefix TD?
Retrieval entering id_9204c751-aa43-4f11-909b-0a60f51cc466_1454_table: TextNode
Retrieving from object TextNode with query What is the alteration category for prefix TD?
Retrieval entering id_9204c751-aa43-4f11-909b-0a60f51cc466_1318_table: TextNode
Retrieving from object TextNode with query What is the alteration category for prefix TD?
Retrieval entering id_9204c751-aa43-4f11-909b-0a60f51cc466_1314_table: TextNode
Retrieving from object TextNode with query What is the alteration category for prefix TD?
Retrieval entering id_9204c751-aa43-4f11-909b-0a60f51cc466_218_table: TextNode
Retrieving from object TextNode with query What is the alteration category for prefix TD?
Retrieval entering id_9204c751-aa43-4f11-909b-0a60f51cc466_214_table: TextNode
Retrieving from object TextNode with query What is the alteration catego

In [23]:
print(response)

The alteration category for prefix TD is "TECHNICAL DIRECTIVE".


In [24]:
response.source_nodes

[NodeWithScore(node=TextNode(id_='id_9204c751-aa43-4f11-909b-0a60f51cc466_1318_table', embedding=None, metadata={'table_df': "{'PREFIX': {0: 'SC', 1: 'SCD', 2: 'SP', 3: 'SW', 4: 'TC', 5: 'TD', 6: 'TDC', 7: 'TEC', 8: 'TMA', 9: 'TRI', 10: 'TZ'}, 'ALTERATION CATEGORY': {0: 'SERVICE CHANGE OR SOFTWARE CHANGE', 1: 'SHIP CHANGE DOCUMENT', 2: 'SPECIAL PROJECTS', 3: 'SOFTWARE DELIVERY', 4: 'TRIDENT COMMAND & CONTROL SYSTEMS', 5: 'TECHNICAL DIRECTIVE', 6: 'TYCOM DISCRETIONARY CHANGE', 7: 'TEMPORARY ENGINEERING CHANGE', 8: 'TRIPER MACHINERY OPERATION', 9: 'TRIDENT SHIP ALTERATION', 10: 'TYPE ZERO ALTERATION'}, '17 Jun 2021': {0: ' ', 1: ' ', 2: ' ', 3: ' ', 4: ' ', 5: ' ', 6: ' ', 7: ' ', 8: ' ', 9: ' ', 10: ' '}}", 'table_summary': 'Table listing different alteration categories with their descriptions.,\nwith the following table title:\nAlteration Categories,\nwith the following columns:\n- PREFIX: Abbreviation for the alteration category\n- ALTERATION CATEGORY: Description of the alteration ca

In [ ]:
contexts = []
for node_with_score in response.source_nodes:
    node = node_with_score.node
    contexts.append(node.text)
print(contexts)

['This table lists different alteration categories along with their corresponding abbreviations.,\nwith the following table title:\nAlteration Category Abbreviations,\nwith the following columns:\n- PREFIX: None\n- ALTERATION CATEGORY: None\n\n|PREFIX|ALTERATION CATEGORY|\n|---|---|\n|TD|TECHNICAL DIRECTIVE|\n|TDC|TYCOM DISCRETIONARY CHANGE|\n|TEC|TEMPORARY ENGINEERING CHANGE|\n|TMA|TRIPER MACHINERY OPERATION|\n|TRI|TRIDENT SHIP ALTERATION|\n|TZ|TYPE ZERO ALTERATION|\n', 'This table lists different alteration categories along with their descriptions.,\nwith the following table title:\nAlteration Categories,\nwith the following columns:\n- PREFIX: Prefix for the alteration category\n- ALTERATION CATEGORY: Description of the alteration category\n- 17 Jun 2021: Additional information\n\n|PREFIX|ALTERATION CATEGORY|17 Jun 2021|\n|---|---|---|\n|SC|SERVICE CHANGE OR SOFTWARE CHANGE| |\n|SCD|SHIP CHANGE DOCUMENT| |\n|SP|SPECIAL PROJECTS| |\n|SW|SOFTWARE DELIVERY| |\n|TC|TRIDENT COMMAND & CON

In [ ]:
query = "What is the telephone number for NSDSA (0310)?"
response = ships_recursive_query_engine.query(query)

Retrieval entering id_5a6a7c8f-743e-4111-bd71-c45b6e37b937_1378_table: TextNode
Retrieving from object TextNode with query What is the telephone number for NSDSA (0310)?
Retrieval entering id_5a6a7c8f-743e-4111-bd71-c45b6e37b937_1380_table: TextNode
Retrieving from object TextNode with query What is the telephone number for NSDSA (0310)?
Retrieval entering id_5a6a7c8f-743e-4111-bd71-c45b6e37b937_1386_table: TextNode
Retrieving from object TextNode with query What is the telephone number for NSDSA (0310)?
Retrieval entering id_5a6a7c8f-743e-4111-bd71-c45b6e37b937_1390_table: TextNode
Retrieving from object TextNode with query What is the telephone number for NSDSA (0310)?
Retrieval entering id_5a6a7c8f-743e-4111-bd71-c45b6e37b937_1374_table: TextNode
Retrieving from object TextNode with query What is the telephone number for NSDSA (0310)?
Retrieval entering id_5a6a7c8f-743e-4111-bd71-c45b6e37b937_1382_table: TextNode
Retrieving from object TextNode with query What is the telephone numbe

In [ ]:
print(response)

The telephone number for NSDSA (0310) is 805-228-0777.


In [ ]:
query = "What is the planned action for job catalog DXCN D701?"
response = ships_recursive_query_engine.query(query)

Retrieval entering id_5a6a7c8f-743e-4111-bd71-c45b6e37b937_1316_table: TextNode
Retrieving from object TextNode with query What is the planned action for job catalog DXCN D701?
Retrieval entering id_5a6a7c8f-743e-4111-bd71-c45b6e37b937_1318_table: TextNode
Retrieving from object TextNode with query What is the planned action for job catalog DXCN D701?
Retrieval entering id_5a6a7c8f-743e-4111-bd71-c45b6e37b937_1100_table: TextNode
Retrieving from object TextNode with query What is the planned action for job catalog DXCN D701?


In [ ]:
print(response)

The planned action for job catalog DXCN D701 is ESM ALIGNMENT.


In [ ]:
query = "What is the repair code for Non-Destructive Testing?"
response = ships_recursive_query_engine.query(query)

Retrieval entering id_5a6a7c8f-743e-4111-bd71-c45b6e37b937_1080_table: TextNode
Retrieving from object TextNode with query What is the repair code for Non-Destructive Testing?
Retrieval entering id_5a6a7c8f-743e-4111-bd71-c45b6e37b937_1072_table: TextNode
Retrieving from object TextNode with query What is the repair code for Non-Destructive Testing?
Retrieval entering id_5a6a7c8f-743e-4111-bd71-c45b6e37b937_1090_table: TextNode
Retrieving from object TextNode with query What is the repair code for Non-Destructive Testing?
Retrieval entering id_5a6a7c8f-743e-4111-bd71-c45b6e37b937_1068_table: TextNode
Retrieving from object TextNode with query What is the repair code for Non-Destructive Testing?
Retrieval entering id_5a6a7c8f-743e-4111-bd71-c45b6e37b937_1084_table: TextNode
Retrieving from object TextNode with query What is the repair code for Non-Destructive Testing?
Retrieval entering id_5a6a7c8f-743e-4111-bd71-c45b6e37b937_1088_table: TextNode
Retrieving from object TextNode with que

In [ ]:
print(response)

93A


## **RAGAS Evaluation**

In [25]:
!pip install langchain

In [26]:
!pip install pypdf

In [27]:
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("./Ships_3m_manual_04790.008D.pdf")
docs = loader.load()

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 700,
    chunk_overlap = 50
)

documents = text_splitter.split_documents(docs)

In [28]:
len(documents)

1152

In [29]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context

generator = TestsetGenerator.with_openai()

testset = generator.generate_with_langchain_docs(documents, test_size=10, distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25})


<ipython-input-29-6bb4ddca3d40>:4: DeprecationWarning: The function with_openai was deprecated in 0.1.4, and will be removed in the 0.2.0 release. Use from_langchain instead.
  generator = TestsetGenerator.with_openai()


embedding nodes:   0%|          | 0/2304 [00:00<?, ?it/s]

Generating:   0%|          | 0/10 [00:00<?, ?it/s]

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [30]:
testset.test_data[0]

DataRow(question='What is one of the specific inspection conditions that can indicate the presence of corrosion?', contexts=['LWC/AWC is to feedback the material condition assessment by "yes/no" responses to conditional questions printed on the CSMP AWR.  In these cases, the narrative will begin with a gener al statement of the requirement, followed by specific inspection conditions and a response \nspace for "Y" or "N."  For example:  "Was there evidence of external leakage? ( );   corrosion?  ( )."'], ground_truth='One of the specific inspection conditions that can indicate the presence of corrosion is evidence of external leakage.', evolution_type='simple', metadata=[{'source': './Ships_3m_manual_04790.008D.pdf', 'page': 263}])

In [31]:
test_df = testset.to_pandas()

In [32]:
test_df

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What is one of the specific inspection conditi...,[LWC/AWC is to feedback the material condition...,One of the specific inspection conditions that...,simple,[{'source': './Ships_3m_manual_04790.008D.pdf'...,True
1,What is the role of CBM methodology in optimiz...,"[maintaining ships, aircraft, and infrastructu...",CBM methodology is designed to optimize life c...,simple,[{'source': './Ships_3m_manual_04790.008D.pdf'...,True
2,What are some alternative test formats that ca...,[(MRC) per reference (b). For equipment and s...,The alternative test formats that can be used ...,simple,[{'source': './Ships_3m_manual_04790.008D.pdf'...,True
3,What is the purpose of Periodic Maintenance Re...,[M reporting. This program was established to...,PMRs provide the capability to generate work r...,simple,[{'source': './Ships_3m_manual_04790.008D.pdf'...,True
4,What is the process for obtaining Classic RCM ...,[a. Backfit RCM Certification. .................,nan,simple,[{'source': './Ships_3m_manual_04790.008D.pdf'...,True
5,How does CBM methodology contribute to optimiz...,"[maintaining ships, aircraft, and infrastructu...",CBM methodology contributes to optimizing life...,reasoning,[{'source': './Ships_3m_manual_04790.008D.pdf'...,True
6,How do conditional questions on CSMP AWR contr...,[LWC/AWC is to feedback the material condition...,nan,multi_context,[{'source': './Ships_3m_manual_04790.008D.pdf'...,True
7,"How does CBM, in conjunction with RCM, optimiz...","[maintaining ships, aircraft, and infrastructu...","CBM, in conjunction with RCM, optimizes aircra...",multi_context,[{'source': './Ships_3m_manual_04790.008D.pdf'...,True
8,What can be gained from the On-line Systems Tr...,[3. Uses of the Data ...........................,The On-line Systems Training Course can provid...,reasoning,[{'source': './Ships_3m_manual_04790.008D.pdf'...,True


In [33]:
test_questions = test_df["question"].values.tolist()
test_groundtruths = test_df["ground_truth"].values.tolist()

In [35]:
file_path_1 = "test_questions.pkl"
file_path_2 = "test_groundtruths.pkl"

# Open the file in binary write mode
with open(file_path_1, 'wb') as file:
    # Dump the object into the file using pickle
    pickle.dump(test_questions , file)

with open(file_path_2, 'wb') as file:
    # Dump the object into the file using pickle
    pickle.dump(test_groundtruths , file)

print("Objects saved successfully!")

Objects saved successfully!


In [52]:
answers = []
contexts = []
count = 0
for question in test_questions:
  response = ships_recursive_query_engine.query(question)
  answers.append(response.response)
  count += 1
  for node_with_score in response.source_nodes:
    node = node_with_score.node
    if len(contexts) < len(answers):
        contexts.append([node.text])

Retrieval entering id_9204c751-aa43-4f11-909b-0a60f51cc466_1012_table: TextNode
Retrieving from object TextNode with query What is one of the specific inspection conditions that can indicate the presence of corrosion?
Retrieval entering id_9204c751-aa43-4f11-909b-0a60f51cc466_1120_table: TextNode
Retrieving from object TextNode with query What is the role of CBM methodology in optimizing life cycle maintenance program costs?
Retrieval entering id_9204c751-aa43-4f11-909b-0a60f51cc466_194_table: TextNode
Retrieving from object TextNode with query What is the purpose of Periodic Maintenance Requirements (PMRs) in the maintenance and material management program?
Retrieval entering id_9204c751-aa43-4f11-909b-0a60f51cc466_1274_table: TextNode
Retrieving from object TextNode with query What is the purpose of Periodic Maintenance Requirements (PMRs) in the maintenance and material management program?
Retrieval entering id_9204c751-aa43-4f11-909b-0a60f51cc466_1486_table: TextNode
Retrieving fro

In [55]:
print(len(contexts))

9


In [56]:
print(len(answers))

9


In [57]:
contexts

[['e. The inspector will enter the data listed in subparagraphs 3.e.(1) through 3.e.(16):\n\n- (1) BLOCK 15, Safety Hazard Code – enter appropriate Risk Assessment Code (RAC).\n- (2) Selected Level Reporting (SLR) – N/A.\n- (3) Remarks – enter observed symptoms and what is wrong, if known, followed by "XXX Troubleshoot & Repair or Replace."\n- (4) Current Ship\'s Maintenance Project (CSMP) Summary – enter summary up to 30 characters.\n- (5) Contacts – select from approved list.\n- (6) EOC\n- (7) Safety – "S"\n- (8) Inspector Name\n- (9) INSURV Department\n- (10) Root Cause Code\n- (11) PRI-RES\n- (12) Reliability/Maint\n- (13) Corrected\n- (14) Corrosion – N/A\n---\n NAVSEAINST 4790.8D\n\n(15) CSMP-worthy\n\n17 Jun 2021\n\n(16) Inspection Status – N/A\n\n Procedures Before INSURV Inspection'],
 ["---\n NAVSEAINST 4790.8D\n\nSECTION V 17 Jun 2021\n\n CHAPTER 1\n\nAIRCRAFT CARRIER, SUBMARINE AND SURFACE SHIP CLASS MAINTENANCE PLANS\n---\n NAVSEAINST 4790.8D\n\n SECTION V | CHAPTER 1\n\n1

In [58]:
from datasets import Dataset

response_dataset = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

In [59]:
response_dataset.features

{'question': Value(dtype='string', id=None),
 'answer': Value(dtype='string', id=None),
 'contexts': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'ground_truth': Value(dtype='string', id=None)}

In [60]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
)

metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
]

In [61]:
results = evaluate(response_dataset, metrics)

Evaluating:   0%|          | 0/45 [00:00<?, ?it/s]

In [62]:
results

{'faithfulness': 0.8571, 'answer_relevancy': 0.9256, 'context_recall': 0.7222, 'context_precision': 0.7778, 'answer_correctness': 0.6010}

In [63]:
results_df = results.to_pandas()
results_df

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,What is one of the specific inspection conditi...,Corrosion Condition.,[e. The inspector will enter the data listed i...,One of the specific inspection conditions that...,0.000000,0.772457,0.0,0.0,0.223765
1,What is the role of CBM methodology in optimiz...,CBM methodology plays a crucial role in optimi...,[---\n NAVSEAINST 4790.8D\n\nSECTION V 17 Jun ...,CBM methodology is designed to optimize life c...,1.000000,0.992323,1.0,1.0,0.879426
2,What are some alternative test formats that ca...,Some alternative test formats that can be used...,[The submarine community does not use APs.\n\n...,The alternative test formats that can be used ...,1.000000,1.000000,1.0,1.0,0.845439
3,What is the purpose of Periodic Maintenance Re...,The purpose of Periodic Maintenance Requiremen...,[---\n NAVSEAINST 4790.8D\n\n APPENDIX E\n\n17...,PMRs provide the capability to generate work r...,1.000000,0.999999,1.0,1.0,0.837786
4,What is the process for obtaining Classic RCM ...,The process for obtaining Classic RCM Certific...,[The table provides information on the require...,nan,1.000000,1.000000,0.0,1.0,0.177771
5,How does CBM methodology contribute to optimiz...,CBM methodology contributes to optimizing life...,[Purpose\n\nThis chapter describes and documen...,CBM methodology contributes to optimizing life...,1.000000,0.877407,1.0,1.0,0.672401
6,How do conditional questions on CSMP AWR contr...,Conditional questions on CSMP AWR contribute t...,[When a single requirement requires a single J...,nan,1.000000,0.969807,0.5,0.0,0.179405
7,"How does CBM, in conjunction with RCM, optimiz...",Repeat,[---\n NAVSEAINST 4790.8D\n\nSECTION V 17 Jun ...,"CBM, in conjunction with RCM, optimizes aircra...",NaN,0.829045,1.0,1.0,0.684994
8,What can be gained from the On-line Systems Tr...,The On-line Systems Training Course provides i...,[---\n NAVSEAINST 4790.8D\n\n(Force Level) equ...,The On-line Systems Training Course can provid...,0.857143,0.889653,1.0,1.0,0.907982
